In [1]:
from pyspark import SparkConf, SparkContext
sc = SparkContext(appName="ClassificationsongLBFGS")

In [2]:
from pyspark.mllib.regression import LabeledPoint
from sklearn import metrics
from numpy import array
import numpy as np
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from pyspark.mllib.classification import LogisticRegressionWithSGD, LogisticRegressionModel,SVMWithSGD, SVMModel,LogisticRegressionWithLBFGS

In [3]:
data = sc.textFile("D://NEU - Big Data and Intelligent Analytics/Midterm/YearPredictionMSD.txt")

In [4]:
#def parsePoint(line):
 #   values = [float(x) for x in line.replace(';', ' ').split(' ')]
  #  return LabeledPoint(values[0], values[1:])

def line_to_fields(line):
    space_separated_line = line.replace(',', ' ')
    string_array = space_separated_line.split(' ')
    float_array = map(float, string_array)
    if float_array[0]<1965.0:
        return LabeledPoint(0.0, float_array[1:])
    else:
        return LabeledPoint(1.0, float_array[1:])

In [5]:
parsedData = data.map(line_to_fields)

In [6]:
from pyspark.mllib.util import MLUtils
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.feature import StandardScaler

label1 = parsedData.map(lambda x: x.label)
features1 = parsedData.map(lambda x: x.features)

scaler2 = StandardScaler(withMean=True, withStd=True).fit(features1)
data2 = label1.zip(scaler2.transform(features1.map(lambda x: Vectors.dense(x.toArray()))))

In [7]:
x = np.array(data2)
x = data2.map(lambda x: x[0])
y = data2.map(lambda x: x[1])
z = data2.map(lambda x: LabeledPoint((x[0]),(x[1])))

#Spliting data into training and test data
(trainingData, testData) = z.randomSplit([0.7, 0.3])

In [8]:
# Build the model
model = LogisticRegressionWithLBFGS.train(trainingData, iterations=100,regParam= 0.1)

In [9]:
# Evaluating the model on training data
labelsAndPreds = trainingData.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_training = labelsAndPreds.filter(lambda (v, p): v == p).count() / float(trainingData.count())
print("Accuracy(training) = " + str(accuracy_training))
x = np.array(labelsAndPreds.collect())

Accuracy(training) = 0.596630478176


In [10]:
# Evaluating the model on test data
labelsAndPreds1 = testData.map(lambda p: (p.label, model.predict(p.features))).cache()
accuracy_test = labelsAndPreds1.filter(lambda (v, p): v == p).count() / float(testData.count())
print("Accuracy(test) = " + str(accuracy_test))
y = np.array(labelsAndPreds1.collect())

Accuracy(test) = 0.594932779793


In [11]:
#clearing the threshold for getting scores as prediction.
model.clearThreshold()

In [12]:
#making an array of true lables and its predicted scores for training data
scoreandlabels = np.array(trainingData.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Training Area under curve  = " + str(roc_auc_score(scoreandlabels[:,0],scoreandlabels[:,1]))
print "Training Area under Precision-Recall = " + str(average_precision_score(scoreandlabels[:,0],scoreandlabels[:,1]))

Training Area under curve  = 0.895549437572
Training Area under Precision-Recall = 0.998177061292


In [13]:
#making an array of true lables and its predicted scores for test data
scoreandlabels1 = np.array(testData.map(lambda p: (p.label, model.predict(p.features))).collect())
print "Test Area under curve  = " + str(roc_auc_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))
print "Test Area under Precision-Recall = " + str(average_precision_score(scoreandlabels1[:,0],scoreandlabels1[:,1]))

Test Area under curve  = 0.893532160068
Test Area under Precision-Recall = 0.998157404516


In [14]:
#Making confusion matrix for training data
confusion_matrix(x[:,0],x[:,1])

array([[  4999,    181],
       [145316, 210208]])

In [15]:
#Making confusion matrix for test data
confusion_matrix(y[:,0],y[:,1])

array([[ 2114,    87],
       [62553, 89887]])